In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **About Dataset**


There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
con = sql.connect('/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db')

In [3]:
query_1 = '''
SELECT *
FROM crime_scene_report
LIMIT 5;
'''

#running our query
pd.read_sql_query(query_1, con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


**Cracking Case:**

Murder Weapon: We know the crime was a murder, so we'll target the type field in the database and look for "murder" cases.

Time of the Crime: Jan 15th, 2018 was the big day, so we'll search the date field for that specific date

.
Location: The crime went down in SQL City, so we'll filter the city field to narrow our search.

In [9]:
#running our first query, looking for data from crime scene report
query_2 = '''
SELECT *
FROM crime_scene_report
WHERE type = 'murder'
AND date = '20180115'
AND city = 'SQL City';
'''
#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)
#running our query
pd.read_sql_query(query_2, con)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [11]:
#supect the first wintness
query_3='''
SELECT* 
FROM person
where address_street_name='Northwestern Dr'
ORDER BY address_number DESC
LIMIT 10;
'''

pd.read_sql(query_3,con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
5,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
6,93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
7,87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
8,36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
9,53076,Boris Bijou,664914,3327,Northwestern Dr,401191868


Witness Found! Our search revealed the first witness is named Morty Schapiro and has an ID number of 14887. We also found an additional ID number, possibly a driver's license, linked to him (118009).

In [13]:
#suspect the 2nd witness
query_4='''
SELECT* 
FROM person
where address_street_name='Franklin Ave'
AND name LIKE '%Annabel%'
'''

pd.read_sql(query_4,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Annabel Found! It looks like we found our second witness! Her name is Annabel Miller and she has an ID number of 16371. We also found an additional ID number, possibly a driver's license, linked to her (490173).

**Witness Statements******

In [14]:
#interview of first witness
query_6='''

SELECT *
FROM interview
where person_id=14887
'''

pd.read_sql(query_6,con)


,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [15]:
#interview of 2nd witness
query_7='''

SELECT *
FROM interview
where person_id=16371
'''

pd.read_sql(query_7,con)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Based on the witness statements, we have a clearer picture of the suspect:
* 
The suspect is a ma
  n* .
He's a member of "Get Fit Now Gym" with a gold membership number starting with "48
  Z* ."
He was at the gym on January 
  9* th.
He drives a car with a license plate containing "H42W."

GET INTO THE GYM

In [16]:
query_8 = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%'
AND check_in_date = '20180109';
'''
#running our query
pd.read_sql_query(query_8, con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [17]:
query_9 = '''
SELECT *
FROM drivers_license
WHERE gender = 'male'
AND plate_number LIKE '%H42W%';
'''
#running our query
pd.read_sql_query(query_9, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [18]:
query_10 = '''
SELECT *
FROM person
WHERE license_id IN (423327, 664760);
'''
#running our query
pd.read_sql_query(query_10, con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [19]:
query_11 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id IN (51739, 67318);
'''
#running our query
pd.read_sql_query(query_11, con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


In [20]:
query_12 = '''
SELECT *
FROM interview
WHERE person_id = 67318;
'''
#running our query
pd.read_sql_query(query_12, con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


New Suspect Profile: Looks like we have a new suspect on our hands! Here's what we know about her:

Female
Height: 5'5" (65") or 5'7" (67")
Hair color: Red
Car: Tesla Model S
Attended the SQL Symphony Concert 3 times in December 2017

In [23]:
query_13 = '''
SELECT *
FROM drivers_license
WHERE gender = 'female'
AND hair_color = 'red'
AND height BETWEEN 65 AND 67
AND car_make = 'Tesla'
AND car_model = 'Model S';
'''
#running our query
pd.read_sql_query(query_13, con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [29]:
query_15 = '''
SELECT *
FROM person
WHERE license_id IN (202298, 291182, 918773);
'''
#running our query
pd.read_sql_query(query_15, con)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [33]:
query_16 = '''
SELECT person_id,
    event_name,
    COUNT(event_name) AS count_event
FROM facebook_event_checkin
WHERE person_id IN (78881, 90700, 99716);
'''
#running our query
pd.read_sql_query(query_16, con)

,person_id,event_name,count_event
0,99716,SQL Symphony Concert,3


In [35]:
query_17 = '''
SELECT id,
    name
FROM  person
WHERE id =99716
'''
#running our query
pd.read_sql_query(query_17, con)

,id,name
0,99716,Miranda Priestly


Our investigation has revealed a shocking twist! All signs point to Miranda Priestly as the mastermind behind the murder!